## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-19-18-52-11 +0000,nypost,Sri Lankan man arrested after exposing himself...,https://nypost.com/2025/11/19/lifestyle/sri-la...
1,2025-11-19-18-50-43 +0000,nypost,Aid chief says UN wants to enter Sudan’s al-Fa...,https://nypost.com/2025/11/19/world-news/aid-c...
2,2025-11-19-18-50-02 +0000,nypost,"Cookie shops are kicking cupcakes to the curb,...",https://nypost.com/2025/11/19/lifestyle/cookie...
3,2025-11-19-18-45-35 +0000,bbc,Israeli strikes across Gaza kill 25 Palestinia...,https://www.bbc.com/news/articles/c62e27j69q9o...
4,2025-11-19-18-45-00 +0000,wsj,Comex Gold Settles 0.40% Higher at $4077.70,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
81,trump,44
168,epstein,29
66,saudi,19
417,new,16
166,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
295,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...,132
312,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,131
253,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...,121
20,2025-11-19-18-01-06 +0000,nyt,Trump Says He Will Approve the Epstein Files R...,https://www.nytimes.com/2025/11/19/us/politics...,111
99,2025-11-19-15-01-08 +0000,nyt,Here’s Who’s Attending Trump’s Dinner for the ...,https://www.nytimes.com/2025/11/18/us/politics...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
295,132,2025-11-18-22-58-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
253,77,2025-11-19-01-33-54 +0000,nypost,Trump announces Saudi Arabia will be designate...,https://nypost.com/2025/11/18/us-news/trump-an...
312,58,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
161,50,2025-11-19-11-26-33 +0000,nypost,White House rips House Democrat’s ‘sick’ defen...,https://nypost.com/2025/11/19/us-news/white-ho...
121,42,2025-11-19-14-04-00 +0000,wsj,EU Proposes Multibillion-Euro Plan to Boost Mi...,https://www.wsj.com/world/europe/eu-proposes-m...
78,39,2025-11-19-15-46-13 +0000,nypost,Harvard to open new inquiry into former presid...,https://nypost.com/2025/11/19/us-news/harvard-...
63,38,2025-11-19-16-32-56 +0000,nypost,"Trump Media stock crashes to all-time lows, wi...",https://nypost.com/2025/11/19/business/trump-m...
155,35,2025-11-19-11-47-57 +0000,nypost,Rep. Clay Higgins explains why he was the only...,https://nypost.com/2025/11/19/us-news/louisian...
207,33,2025-11-19-07-01-00 +0000,wsj,Elon Musk’s xAI Is in Advanced Talks to Raise ...,https://www.wsj.com/tech/ai/elon-musks-xai-in-...
244,32,2025-11-19-02-17-48 +0000,nypost,Panera Bread launches multimillion-dollar turn...,https://nypost.com/2025/11/18/business/panera-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
